In [1]:
import dotenv
from agents import Agent, Runner, trace, function_tool

import chromadb

dotenv.load_dotenv()

True

In [2]:
chroma_client = chromadb.PersistentClient(path="../chroma")
calories_db = chroma_client.get_collection(name="nutrition_db")
nutrition_qna_db = chroma_client.get_collection(name="nutrition_qna")

In [3]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function to look up calorie information.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = calories_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)


@function_tool
def nutrition_qna_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function to ask a question about nutrition.

    Args:
        query: The question to ask.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the question and the answer related to the query.
    """

    results = nutrition_qna_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)

    return "Related answers to your question:\n" + "\n".join(formatted_results)

In [4]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information and nutrtion advice    .
    You give concise answers.
    
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the nutrition_qna_tool first to see if there is an answer in the knowledge base.
    """,
    tools=[calorie_lookup_tool, nutrition_qna_tool]
)

In [5]:
with trace("Nutrition Assistant with Nutrition and Calorie RAG"):
    result = await Runner.run(
        calorie_agent,
        "What are the best meal choices for pregnant women and how many calories do they have?",
    )
    print(result.final_output)

Here are concise, evidence-aligned ideas:

Key focus
- Aim for a nutrient-dense, balanced diet with around 200 extra calories per day during pregnancy (varies by trimester and doctor’s advice).
- Emphasize iron, folate, calcium, iodine, protein, fiber, and DHA.

Best meal choices (by nutrients)
- Protein-rich foods: lean meats, fish (low-mercury), eggs, dairy, tofu, legumes.
- Iron-rich options: lean beef, chicken, lentils, spinach, fortified cereals; pair with vitamin C sources to improve absorption.
- Folate-friendly: leafy greens, beans, citrus, fortified grains.
- Calcium sources: yogurt, milk, cheese, fortified plant milks, cheese, leafy greens.
- DHA sources: fatty fish (salmon, sardines) or a supplement if advised.
- Whole grains, fruits, vegetables, and hydration (water/milk).

Sample day (rough calories 1,800–2,100 total; add ~200 kcal over baseline as advised)
- Breakfast: Greek yogurt with berries and oats (about 350 kcal)
- Snack: Apple with peanut butter (about 150–200 kca